# Tuning Fufi hyperparameters with W&B Sweep 🧹 🐶
The idea is automatically tuning hyperparameters with the sweep by the means of a `grid `search.
In order not to spend an eternity doing this thing, we start with a` random` search, and then put a `grid `search on it.

## W&B Setup
🪄 Install `wandb` library and login

Start by installing the library and logging in to your free account.

In [1]:
!pip install wandb -qU
# Log in to your W&B account
import wandb
wandb.login()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.1 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Installing libraries 📚

In [1]:
!pip install gym==0.25.2
#needed from March
!pip install numpy==1.23.5

In [2]:
!pip install stable_baselines3 -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 30.4 MB/s eta 0:00:00


## Setting things up for the environment 🌍 🪖

In [3]:
#cloning Fufi repo from git
!git clone https://github.com/Gaianeve/gym-TheRealFufi.git
#installing things
!pip install /content/gym-TheRealFufi

Cloning into 'gym-TheRealFufi'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 39 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 15.16 KiB | 7.58 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Processing ./gym-TheRealFufi
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
# Enter the environment directory
%cd /content/gym-TheRealFufi
# Actually importing the library for our environment
import gym_TheRealFufi

/content/gym-TheRealFufi


In [5]:
#get back to content directory so I save everything there
%cd ..
!pwd

/content
/content


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## importing libraries and functions 📚


In [6]:
#libraries
import argparse
import random
import time
from distutils.util import strtobool
import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

## Log needed function from files 📡
Loading files directly from git, so I don't have to upload it by hand.

In [7]:
#get files from git
!git clone https://github.com/Gaianeve/The_real_Fufi.git

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cloning into 'The_real_Fufi'...
remote: Enumerating objects: 493, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 493 (delta 132), reused 203 (delta 96), pack-reused 204 (from 1)
Receiving objects: 100% (493/493), 7.54 MiB | 9.97 MiB/s, done.
Resolving deltas: 100% (223/223), done.


In [8]:
!pwd
%cd The_real_Fufi/Software
from environment import vectorize_env
from agent_class import Agent
from agent_utils import anneal, collect_data, GAE, PPO_train_agent, evaluate_agent


/content
/content/The_real_Fufi/Software


In [9]:
#back to content directory just to be sure
%cd ..
%cd ..

/content/The_real_Fufi
/content


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
#import main function from file
!pwd
%cd The_real_Fufi/Software
from sweep_main import train_main, parse_args, get_default_hyperparameters_and_names

#back to content directory
%cd ..
%cd ..

/content
/content/The_real_Fufi/Software
/content/The_real_Fufi
/content


/usr/local/lib/python3.10/dist-packages/wandb/analytics/sentry.py:90: SentryHubDeprecationWarning: `sentry_sdk.Hub` is deprecated and will be removed in a future major release. Please consult our 1.x to 2.x migration guide for details on how to migrate `Hub` usage to the new API: https://docs.sentry.io/platforms/python/migration/1.x-to-2.x
  self.hub = sentry_sdk.Hub(client)


## Define the sweep 📑
Standard default parameters from *stable baseline* work pretty well, so I guess a vanilla random, search with the valus that i give him, would be more than enough. I move in the range i see on the internet for a continous PPO with gSDE

*   `beta ` ➡ in [0.2,0.4] at a pace of 0.02
*  ` num_envs`  ➡ 4,8,16
*   `learning_rate` ➡ in [1e**-4, 3e**-4] at a pace 0.2
*  ` num_steps`  ➡ in [128, 2048] by powers of 2
*  ` num_minibatch`  ➡ in [4, 32] by powers of 2

In [15]:
import numpy as np
import random

# Define sweep config
sweep_configuration = {
    "method": "random",
    "name": "sweep_The_Real_Fufi",
    "metric": {"goal": "maximize", "name": "sum_episodes"},
    "parameters": {
       "num_steps": {"values": [128, 256, 512, 2048]},
       "num_envs" : {"values": [2,4,8,16]},
       "beta" : {"values": [0.20, 0.22, 0.24, 0.26, 0.28, 0.30, 0.32, 0.34, 0.36, 0.38, 0.40]},
       "learning_rate":  {"values": [0.0001, 0.00012,0.00014, 0.00016,0.00018,0.00020, 0.00022, \
                                     0.00024, 0.00026, 0.00028, 0.00030]},
       "num_minibatches": {"values": [4, 8, 16, 32]},
    },
}


In [16]:
#print the result
import pprint
pprint.pprint(sweep_configuration)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'sum_episodes'},
 'name': 'sweep_The_Real_Fufi',
 'parameters': {'beta': {'values': [0.2,
                                    0.22,
                                    0.24,
                                    0.26,
                                    0.28,
                                    0.3,
                                    0.32,
                                    0.34,
                                    0.36,
                                    0.38,
                                    0.4]},
                'learning_rate': {'values': [0.0001,
                                             0.00012,
                                             0.00014,
                                             0.00016,
                                             0.00018,
                                             0.0002,
                                             0.00022,
                                             0.00024,

## Run main with the sweep 🏃 🧹
The `wandb.sweep` function initializes the sweep using the configuration. The `wandb.agent `function runs the sweep, executing the `sweep_main` function for each set of parameters.

📚 **Handling Parameters in Script**: In `sweep_main`, `wandb.init()` initializes a run. The script updates the args with the parameters from the sweep `(wandb.config)`, which are then passed to the main function.

📚 **Note**: Added `if __name__ == "__main__":` This ensures that main is called only when the script is executed directly, not when imported as a module.

In [18]:
import wandb
if __name__ == "__main__":
    sweep_id = wandb.sweep(sweep=sweep_configuration, project="The_Real_Fufino_sweep")  # Set up the sweep

    #get default configuration
    default_config  = get_default_hyperparameters_and_names()

    def sweep_train():
      print(default_config)
      parse_args(default_config)
      train_main(default_config)


Create sweep with ID: vj6gktsx
Sweep URL: https://wandb.ai/cartpole_maria_gaia/The_Real_Fufino_sweep/sweeps/vj6gktsx


In [ ]:
 wandb.agent(sweep_id, function=sweep_train, count = 50)

wandb: Agent Starting Run: 6vqbnipr with config:
wandb: 	beta: 0.4
wandb: 	learning_rate: 0.0002
wandb: 	num_envs: 16
wandb: 	num_minibatches: 8
wandb: 	num_steps: 2048


namespace(exp_name='The_Real_Fufi_adventures', gym_id='TheRealFufi-v0', torch_deterministic=True, cuda=True, wandb_project_name='The_Real_Fufino_sweep', wandb_entity=None, capture_video=True, beta=0.3, lr=0.00025, seed=1, total_timesteps=1000000, num_envs=4, num_steps=128, anneal_lr=True, gae=True, gamma=0.99, gae_lambda=0.95, num_minibatches=4, update_epochs=10, norm_adv=True, clip_coef=0.2, clip_vloss=True, ent_coef=0.01, vf_coef=0.5, max_grad_norm=0.5, target_kl=None)


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
/usr/local/lib/python3.10/dist-package